In [1]:
!pip install sodapy

  Using cached https://files.pythonhosted.org/packages/69/12/ce6d8c29b7e60a42b1128998169ba0b10e8c19e3ea4a4387526d167c57ef/sodapy-1.4.6-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/00/2b/8d082ddfed935f3608cc61140df6dcbf0edea1bc3ab52fb6c29ae3e81e85/future-0.16.0.tar.gz
  Using cached https://files.pythonhosted.org/packages/49/df/50aa1999ab9bde74656c2919d9c0c085fd2b3775fd3eca826012bef76d8c/requests-2.18.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/7c/e6/92ad559b7192d846975fc916b65f667c7b8c3a32bea7372340bfe9a15fa5/certifi-2018.4.16-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/bc/a9/01ffebfb562e4274b6487b4bb1ddec7ca55ec7510b22e4c51f14098443b8/chardet-3.0.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/27/cc/6dd9a3869f15c2edfab863b992838277279ce92663d334df9ecf5106f5c6/idna-2.6-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/63/cb/6965947c13a

Command "/usr/bin/python -u -c "import setuptools, tokenize;__file__='/private/var/folders/w0/rw_vpp8n61l79d6045kc0khm0000gn/T/pip-install-69K1nP/future/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record /private/var/folders/w0/rw_vpp8n61l79d6045kc0khm0000gn/T/pip-record-q7A4ux/install-record.txt --single-version-externally-managed --compile" failed with error code 1 in /private/var/folders/w0/rw_vpp8n61l79d6045kc0khm0000gn/T/pip-install-69K1nP/future/


In [2]:
# from https://github.com/socrata/dev.socrata.com/blob/39c6581986466edb5e7f72f5beea5ce69238f8de/snippets/pandas.py

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofchicago.org", None)

# First 50000 results, returned as JSON from API 
# Connverted to Python list of dictionaries by sodapy.
# Column names converted to snake case, special chars removed
results = client.get("r5kz-chrr", limit=50000)

# Convert to pandas DataFrame
business_licenses = pd.DataFrame.from_records(results)

In [3]:
# download remaining business licenses (limit 50000 / call)
start = 50000
while results:
    print(start)
    results = client.get("r5kz-chrr", limit=50000, offset=start)
    business_licenses = business_licenses.append(pd.DataFrame.from_records(results))
    start += 50000


50000
100000
150000
200000
250000
300000
350000
400000
450000
500000


/usr/local/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


550000
600000
650000
700000
750000
800000
850000
900000
950000


In [9]:
type(business_licenses.iloc[40].application_requirements_complete)


str

In [5]:
business_licenses.columns


Index(['account_number', 'address', 'application_created_date',
       'application_requirements_complete', 'application_type',
       'business_activity', 'business_activity_id', 'city',
       'conditional_approval', 'date_issued', 'doing_business_as_name',
       'expiration_date', 'id', 'latitude', 'legal_name',
       'license_approved_for_issuance', 'license_code', 'license_description',
       'license_id', 'license_number', 'license_start_date', 'license_status',
       'license_status_change_date', 'location', 'longitude', 'payment_date',
       'police_district', 'precinct', 'site_number', 'ssa', 'state', 'ward',
       'ward_precinct', 'zip_code'],
      dtype='object')

In [7]:
business_licenses.to_csv("../../DATA/business_licenses")